<div style="text-align: center;">
  <h1>Smart beta ETF</h1>
  <h3>Equally Risk-Contributed Portfolio</h3>
</div>

# Theory on Equally Risk-Contributed portfolios
An Equally Risk-Contributed (ERC) portfolio is a portfolio construction method where each asset contributes equally to the total portfolio risk. Unlike traditional equal-weighted portfolios (where each asset has the same allocation), ERC portfolios focus on balancing risk contributions, not just weights.

An ERC portfolio provides:
* Better diversification: ERC portfolios often achieve better risk-adjusted returns compared to equal-weighted or market-cap-weighted portfolios, especially in volatile or uncertain markets.
* Risk control: By explicitly managing risk contributions, ERC portfolios avoid concentration in a few high-risk assets.
* Robustness: ERC portfolios are less sensitive to estimation errors in expected returns, making them more robust in practice.


The total risk of a portfolio is considered to be the standard deviation of its returns: $\sigma_P=\sqrt{w^T\Sigma w}$ where $\Sigma$ is the covariance matrix of the asset returns and $w$ is the vector of weights.\
We define the marginal risk contribution (MRC) as the derivative of portfolio risk with respect to an asset's weight: 
$$\text{MRC}_i=\frac{\partial \sigma_P}{\partial w_i}$$
The risk contributions are then defined as $\text{RC}_i = w_i \text{MRC}_i$ such that $\sum_i \text{RC}_i = \sigma_P$.\
In an ERC portfolio, the risk contribution of each asset is equal:
$$\text{RC}_i = \frac{\sigma_P}{N}$$
where $N$ is the number of assets.\
The ERC portfolio is constructed by solving an optimization problem:
$$\min_w \sum_i \left( \text{RC}_i - \frac{\sigma_P}{N} \right)^2$$
subject to $\sum_i w_i=1, \quad w_i \geq 0$.\

More details on ERC portfolios can be found in the [paper of Maillard, Roncalli and Teiletche](http://thierry-roncalli.com/download/erc.pdf).

# Implementation
We have chosen to create an S&P 500 ERC portfolio where classical factors (Quality, Value, Dividends, Momentum, Low Volatility) have equal risk contributions. These factors may be approximated by ETFs. 

The first part of our work has been to gather financial data required to construct some ETFs (Quality, Value, Dividends) using AlphaVantage's API and then to construct these ETFs. This part has been done by my classmate and this notebook focuses on my contribution to the project. Thus, I take as input the files containing prices and weights of each ETF.

## Libraries

In [56]:
# pip install yfinance plotly PyPortfolioOpt nbformat openpyxl lxml

In [57]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as opt
import json
import plotly.graph_objects as go
import requests
from io import BytesIO, StringIO
import pypfopt as pf

## Portfolio construction

In [ ]:
def transaction_costs(w, w0, TC):
    """Calculate the cost of transactions between two sets of portfolio weights.

    Parameters
    ----------
    w : array_like
        New portfolio weights.
    w0 : array_like
        Previous portfolio weights.
    TC : array_like
        Transaction costs per asset (e.g., 5 bps or 0.0005).

    Returns
    -------
    float
        Total transaction cost.
    """
    return np.sum(TC * np.abs(w - w0)**2)

def objective(w, cov_mat, w0, TC, factor_TC):
    """Objective function for ERC optimization with transaction costs.

    Parameters
    ----------
    w : array_like
        Current portfolio weights.
    cov_mat : array_like
        Covariance matrix of returns.
    w0 : array_like
        Previous portfolio weights.
    TC : array_like
        Transaction costs per asset.
    factor_TC : float
        Factor to adjust the importance of transaction costs in the objective function.

    Returns
    -------
    float
        Value of the objective function.
    """
    # Original variance-based function
    s = 0
    for i in range(len(w)):
        for j in range(len(w)):
            s += (w[i] * (cov_mat @ w)[i] - w[j] * (cov_mat @ w)[j])**2
    # Add transaction costs as a penalty term
    return s + factor_TC * transaction_costs(w, w0, TC)

def erc_weights(df, w0=np.array([0.2, 0.2, 0.2, 0.3, 0.1]), TC=np.array([0.0005, 0.0005, 0.0005, 0.0005, 0.0005]), factor_TC=0.0001):
    """Calculate the weights for the risk parity portfolio.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with the ETF returns.
    w0 : array_like, optional
        Initial weights (current portfolio). Default is [0.2, 0.2, 0.2, 0.3, 0.1].
    TC : array_like, optional
        Transaction costs (5 bps). Default is [0.0005, 0.0005, 0.0005, 0.0005, 0.0005].
    factor_TC : float, optional
        Factor to adjust the importance of transaction costs in the objective function. Default is 0.0001.

    Returns
    -------
    array_like
        Optimized portfolio weights.
    """
    cov_mat = df.cov().values  # Covariance matrix of ETF returns
    # Constraints: weights sum to 1 and are long-only
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum of weights = 1
        {'type': 'ineq', 'fun': lambda w: w}            # Weights must be positive (long only)
    ]
    # Optimize
    result = opt.minimize(objective, w0, args=(cov_mat, w0, TC, factor_TC), constraints=constraints, tol=1e-20)
    w_opt = result.x  # Optimized weights
    return w_opt

def min_var_weights(train_data):
    """Calculate the weights for the minimum variance portfolio.

    Parameters
    ----------
    train_data : pandas.DataFrame
        DataFrame with the training returns.

    Returns
    -------
    array_like or None
        Optimized weights for the minimum variance portfolio. Returns None if an error occurs.
    """
    try:
        mean_returns = train_data.mean()
        cov_matrix = train_data.cov()
        ef = pf.EfficientFrontier(mean_returns, cov_matrix)
        weights = ef.min_volatility()
        return np.array(list(weights.values()))
    except Exception as e:
        print(f"Error calculating min-var portfolio weights: {e}")
        return None

## Performance metrics computation

In [59]:
def calculate_metrics(portfolio_returns, spy_returns, slice):
    """Calculate performance metrics for a given portfolio returns for a specific time slice.

    Parameters
    ----------
    portfolio_returns : pandas.Series
        Returns of the portfolio.
    spy_returns : pandas.Series
        Returns of the benchmark (e.g., SPY).
    slice : int
        Identifier for the time slice.

    Returns
    -------
    dict or None
        Dictionary of performance metrics. Returns None if an error occurs.
    """
    try:
        cumulative_return = float((1 + portfolio_returns).prod() - 1)
        cov_matrix = np.cov(portfolio_returns, spy_returns)
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        expected_annual_return = float(portfolio_returns.mean() * 252)
        volatility = float(portfolio_returns.std() * np.sqrt(252))
        sharpe_ratio = float(expected_annual_return / volatility)
        treynor_ratio = float(cumulative_return / beta)
        return {
            "Slice": slice,
            "Expected Annual Return": expected_annual_return,
            "Volatility": volatility,
            "Sharpe Ratio": sharpe_ratio,
            "Treynor Ratio": treynor_ratio,
            "Cumulative Return": cumulative_return
        }
    except Exception as e:
        print(f"Error calculating metrics for slice {slice}: {e}")
        return None

## ETFs data

In [60]:
def get_data_from_etf_files(start_date, end_date, etf_files):
    """Retrieve ETF price data from files and Yahoo Finance (for SPY), then calculate returns.

    Parameters
    ----------
    start_date : str
        Start date for data retrieval (format: 'YYYY-MM-DD').
    end_date : str
        End date for data retrieval (format: 'YYYY-MM-DD').
    etf_files : dict
        Dictionary mapping ETF names to their respective file paths.

    Returns
    -------
    tuple of (pandas.Series, pandas.DataFrame) or None
        - spy_returns: SPY returns as a pandas Series.
        - etf_returns: DataFrame of ETF returns for each ETF in `etf_files`.
        Returns None if an error occurs.
    """
    try:
        # Retrieve SPY data from Yahoo Finance
        spy_data = yf.download('SPY', start=start_date, end=end_date)['Close']
        spy_returns = spy_data.pct_change().dropna()

        # Retrieve ETF data from files
        etf_returns = {}
        for etf_name, file_path in etf_files.items():
            with open(file_path, 'r') as f:
                etf_data = json.load(f)

            # Formatting
            etf_prices = pd.Series(etf_data, name=etf_name)
            etf_prices.index = pd.to_datetime(etf_prices.index)

            # Filtering
            etf_prices = etf_prices.loc[start_date:end_date]

            # Calculating returns
            etf_return = etf_prices.pct_change().dropna()
            etf_returns[etf_name] = etf_return

        etf_returns = pd.DataFrame(etf_returns)
        return spy_returns, etf_returns
    except Exception as e:
        print(f"Error retrieving or processing data: {e}")
        return None, None

In [ ]:
etf_files = {
    'ETF_div': './data_smart_beta/SP500DividendETFPrices.json',
    'ETF_low_vol': './data_smart_beta/SP500LowVolETFPrices.json',
    'ETF_quality': './data_smart_beta/SP500QualityETFPrices.json',
    'ETF_momentum': './data_smart_beta/SP500MomentumETFPrices.json',
    'ETF_MKT': './data_smart_beta/SP500MarketCapETFPrices.json',
}

spy_returns, etf_returns = get_data_from_etf_files(start_date='2020-01-01',
                                                   end_date='2024-11-29',
                                                   etf_files=etf_files)

/tmp/ipykernel_36101/1949059465.py:22: FutureWarning:

YF.download() has changed argument auto_adjust default to True



[*********************100%***********************]  1 of 1 completed


## Backtest

In [ ]:
def convert_metrics(metrics_list, date_list):
    """Convert a list of performance metrics into a pandas DataFrame.

    Parameters
    ----------
    metrics_list : list of dict
        List of dictionaries, where each dictionary contains performance metrics for a time slice.
    date_list : list of str
       List of dates corresponding to each time slice.

    Returns
    -------
    pandas.DataFrame or None
        DataFrame of metrics indexed by date. Returns None if an error occurs.
    """
    try:
        metrics_df = pd.DataFrame(metrics_list)
        metrics_df.index = pd.to_datetime(date_list)
        return metrics_df
    except KeyError as e:
        print(f"KeyError during DataFrame conversion: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

def backtest(i, j, start_date, end_date, etf_files, strategy):
    """Perform backtesting for a given strategy and return metrics and weights.

    Parameters
    ----------
    i : int
        Length of each time slice (in periods).
    j : int
        Length of the rolling optimization window (in periods).
    start_date : str
        Start date for backtesting (format: 'YYYY-MM-DD').
    end_date : str
        End date for backtesting (format: 'YYYY-MM-DD').
    etf_files : dict
        Dictionary mapping ETF names to their respective file paths.
    strategy : str
        Strategy to backtest ('ERC', 'Equal weight', 'Benchmark', or 'Min-var').

    Returns
    -------
    tuple of (pandas.DataFrame, list) or None
        - metrics_df: DataFrame of performance metrics for each time slice.
        - weights_list: List of optimized weights for each time slice.
        Returns None if an error occurs.
    """
    try:
        if j < i:
            raise ValueError("Optimization window length (j) must be greater than time slice length (i).")

        spy_returns, etf_returns = get_data_from_etf_files(start_date, end_date, etf_files)
        if spy_returns is None or etf_returns is None:
            return None

        train_data = etf_returns.iloc[:j, :]
        test_data = etf_returns.iloc[j:, :]
        spy_test_data = spy_returns.iloc[j:]
        metrics_list, weights_list, date_list = [], [], []

        start_idx = 0
        while start_idx + i <= len(test_data) and start_idx + i <= len(spy_test_data):
            # Slicing of test data
            slice_data = test_data.iloc[start_idx:start_idx + i]
            spy_slice_data = spy_test_data.iloc[start_idx:start_idx + i].to_numpy().flatten()

            # Calculation of weights and portfolio returns
            if strategy == "ERC":
                weights = erc_weights(train_data)
                portfolio_returns = slice_data @ weights
            elif strategy == "Equal weight":
                weights = np.array([1 / len(etf_files)] * len(etf_files))
                portfolio_returns = slice_data @ weights
            elif strategy == "Benchmark":
                portfolio_returns = spy_slice_data
                weights = None  # No weights for benchmark
            elif strategy == "Min-var":
                weights = min_var_weights(train_data)
                portfolio_returns = slice_data @ weights
            else:
                print(f"Invalid strategy: {strategy}")
                return None

            if weights is not None:
                weights_list.append(weights)

            # Metrics calculation
            metrics_list.append(calculate_metrics(portfolio_returns, spy_slice_data, len(metrics_list) + 1))
            date_list.append(test_data.index[start_idx + i])

            # Update training data
            train_data = pd.concat([train_data, test_data.iloc[start_idx:start_idx + i]]).iloc[-j:]
            start_idx += i

        metrics_df = convert_metrics(metrics_list, date_list)
        return metrics_df, weights_list
    except ValueError as e:
        print(f"ValueError: {e}")
        return None
    except Exception as e:
        print(f"Error during backtest for strategy {strategy}: {e}")
        return None

In [86]:
# Initialisation
start_date = '2020-01-01'
end_date = '2024-11-29'
i = 5
j = 22
strategies = ["Equal weight", "Benchmark", "Min-var", "ERC"]
sharpe_ratios = pd.DataFrame(columns=strategies)
cumulative_returns = pd.DataFrame(columns=strategies)

# Backtest
for strategy in strategies:
    metrics_df, weights_list = backtest(i, j, start_date, end_date, etf_files, strategy)
    if metrics_df is not None:
        sharpe_ratios[strategy] = metrics_df["Sharpe Ratio"]
        cumulative_returns[strategy] = metrics_df["Cumulative Return"]
    else:
        print(f"Backtest failed for strategy {strategy}")

/tmp/ipykernel_36101/1949059465.py:22: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_36101/1949059465.py:22: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_36101/1949059465.py:22: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_36101/1949059465.py:22: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [87]:
sharpe_ratios.describe()

,Equal weight,Benchmark,Min-var,ERC
count,242.000000,242.000000,242.000000,242.000000
mean,2.180055,2.986952,2.226074,2.175772
std,9.911489,10.618586,10.119053,9.864649
min,-33.922874,-27.638130,-42.794718,-33.499806
25%,-3.981299,-3.455574,-2.971268,-3.689929
50%,2.496403,2.057053,2.387442,2.378730
75%,7.957431,8.349732,7.676264,7.892352
max,40.556733,46.408368,48.286374,39.461308


In [88]:
def plot_data(data):
    """Plot time series data for each column in the DataFrame.

    This function creates a line plot for each column in the input DataFrame,
    allowing for visual comparison of trends over time.

    Parameters
    ----------
    data : pandas.DataFrame
        DataFrame where each column represents a time series to be plotted.
        The index should be a datetime or numeric index.

    Returns
    -------
    None
        Displays an interactive line plot using Plotly.
    """
    # Create a figure
    fig = go.Figure()

    # Loop through each column in the DataFrame and add a trace for each
    for column in data.columns:
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data[column],
                mode='lines',
                name=column,
                hovertemplate='%{y:.4f}'
            )
        )

    # Update layout for better visualization
    fig.update_layout(
        title="Time Series Trends for Each Column",
        xaxis_title="Time",
        yaxis_title="Value",
        legend_title="Columns",
        hovermode="x unified",
    )

    # Show the plot
    fig.show()

def box_plot_data(data):
    """Create a box plot for each column in the DataFrame.

    This function visualizes the distribution of values for each column,
    highlighting median, quartiles, and potential outliers.

    Parameters
    ----------
    data : pandas.DataFrame
        DataFrame where each column represents a variable to be plotted.

    Returns
    -------
    None
        Displays an interactive box plot using Plotly.
    """
    fig = go.Figure()

    for column in data.columns:
        fig.add_trace(
            go.Box(
                y=data[column],
                name=column,
                boxpoints="all",  # Show all points
                jitter=0.3,       # Spread points for better visibility
                pointpos=-1.8,    # Position points relative to the box
                hovertemplate='%{y:.4f}'
            )
        )

    # Update layout for better visualization
    fig.update_layout(
        title="Distribution of Values for Each Column",
        yaxis_title="Values",
        xaxis_title="Columns",
        legend_title="Columns",
        showlegend=False,  # Box plots are self-explanatory; legend is redundant
    )

    fig.show()

def plot_moving_average_data(data, window=30):
    """Plot smoothed trends using a moving average for each column in the DataFrame.

    This function applies a rolling mean to smooth the data and plots the results,
    making it easier to identify long-term trends.

    Parameters
    ----------
    data : pandas.DataFrame
        DataFrame where each column represents a time series to be smoothed.
    window : int, optional
        Size of the rolling window for the moving average. Default is 30.

    Returns
    -------
    None
        Displays an interactive line plot of smoothed trends using Plotly.
    """
    # Apply a rolling mean (moving average) for smoothing
    smoothed_data = data.rolling(window=window, min_periods=1).mean()

    fig = go.Figure()

    for column in smoothed_data.columns:
        fig.add_trace(
            go.Scatter(
                x=smoothed_data.index,
                y=smoothed_data[column],
                mode='lines',
                name=f"{column} (MA {window})",
                hovertemplate='%{y:.4f}'
            )
        )

    # Update layout for better visualization
    fig.update_layout(
        title=f"Smoothed Trends (Moving Average, Window={window})",
        xaxis_title="Time",
        yaxis_title="Smoothed Values",
        legend_title="Columns",
        hovermode="x unified",
    )

    fig.show()


In [89]:
plot_data(sharpe_ratios)
box_plot_data(sharpe_ratios)
plot_moving_average_data(sharpe_ratios)

In [99]:
def plot_cumulative_returns(
    cumulative_returns_ptf: pd.DataFrame,
    etf_returns: pd.DataFrame,
    title: str = "Cumulative Returns Comparison"
) -> None:
    """Plot cumulative returns for min-var, ERC, individual ETFs, and an optional benchmark.

    Parameters
    ----------
    cumulative_returns_ptf : pd.DataFrame
        Cumulative returns for the minimum variance, equally risk-contributed (ERC) and benchmark (e.g., SPY) portfolios.
    etf_returns : pd.DataFrame
        DataFrame of raw returns for individual ETFs (columns are ETFs, index is date).
    title : str, optional
        Title for the plot. Default is "Cumulative Returns Comparison".

    Returns
    -------
    None
        Displays an interactive plot of cumulative returns using Plotly.
    """
    # Calculate cumulative returns for individual ETFs
    cumulative_etf_returns = (1 + etf_returns).cumprod() - 1
    cumulative_returns_ptf = (1 + cumulative_returns_ptf).cumprod() - 1

    # Create a figure
    fig = go.Figure()

    # Add traces for min-var, ERC and benchmark portfolios
    for column in cumulative_returns_ptf.columns:
        fig.add_trace(
            go.Scatter(
                x=cumulative_returns_ptf.index,
                y=cumulative_returns_ptf[column],
                name=f"{column}",
                mode="lines",
                line=dict(width=1),
            )
        )

    # Add traces for individual ETFs
    for etf in cumulative_etf_returns.columns[1:]:
        fig.add_trace(
            go.Scatter(
                x=cumulative_etf_returns.index,
                y=cumulative_etf_returns[etf],
                name=f"{etf}",
                mode="lines",
                line=dict(width=1),
            )
        )

    # Update layout for better visualization
    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Cumulative Returns",
        legend_title="Portfolio/ETF",
        hovermode="x unified",
        template="plotly_white",
    )

    # Show the plot
    fig.show()

In [100]:
plot_cumulative_returns(
    cumulative_returns,
    etf_returns,
    title="Cumulative Returns: Portfolios vs. ETFs"
)

## From ETFs weights to stocks weights

In [ ]:
def download_data(url):
    """Download and return data from a URL, handling Excel and CSV files appropriately.

    This function downloads data from a given URL, detects the file type (Excel or CSV),
    and parses it into a pandas DataFrame. It skips irrelevant rows and handles common
    CSV parsing issues (e.g., different delimiters or headers).

    Parameters
    ----------
    url : str
        URL pointing to the data file (Excel or CSV).

    Returns
    -------
    pandas.DataFrame
        Parsed DataFrame containing the data from the URL.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()

        if url.endswith(('.xls', '.xlsx')):
            skip_rows = 4
            data = pd.read_excel(BytesIO(response.content), skiprows=skip_rows)
        else:
            skip_rows = 0
            csv_data = response.content.decode('utf-8')
            try:
                data = pd.read_csv(StringIO(csv_data), skiprows=skip_rows)
            except pd.errors.ParserError:
                try:
                    data = pd.read_csv(StringIO(csv_data), delimiter=';', skiprows=skip_rows)
                except pd.errors.ParserError:
                    data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)

        # Ensure the data contains a 'weight' column
        if not any('weight' in col.lower() for col in data.columns):
            data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)

        return data
    except Exception as e:
        print(f"Error downloading or parsing data from {url}: {e}")
        return pd.DataFrame()

def normalize_ticker(ticker):
    """Normalize a ticker symbol by removing spaces and unnecessary characters.

    Parameters
    ----------
    ticker : str or pandas.NA
        Raw ticker symbol, which may contain spaces or special characters.

    Returns
    -------
    str
        Normalized ticker symbol in uppercase, with no spaces or special characters.
    """
    if pd.isna(ticker):
        return ''
    return ticker.replace('/', '').strip().upper()

def calculate_total_weights_per_etf(url, etf_weight):
    """Calculate the total weight for each stock in an ETF from a CSV or Excel file.

    This function downloads the ETF holdings data, identifies the weight and ticker columns,
    normalizes the tickers, and calculates the weighted contribution of each stock to the ETF.

    Parameters
    ----------
    url : str
        URL pointing to the ETF holdings file (CSV or Excel).
    etf_weight : float
        Weight of the ETF in the overall portfolio.

    Returns
    -------
    pandas.DataFrame or None
        DataFrame with columns 'Ticker', 'Name', and 'Total_Weight'.
        Returns None if the weight or ticker column is not found.
    """
    data = download_data(url)
    if data.empty:
        return None

    # Identify weight and ticker columns
    weight_column = next((col for col in data.columns if 'weight' in col.lower()), None)
    if weight_column is None:
        print("Column 'Weight' or 'Weight (%)' not found in the file.")
        return None

    holding_ticker_column = next((col for col in data.columns if 'holding ticker' in col.lower()), None)
    ticker_column = next((col for col in data.columns if 'ticker' in col.lower()), None)

    if holding_ticker_column:
        ticker_column = holding_ticker_column
    elif not ticker_column:
        print("No 'Ticker' or 'Holding Ticker' column found in the file.")
        return None

    # Filter out non-stock holdings
    if 'Name' in data.columns:
        data = data[~data['Name'].isin([
            "US DOLLAR", "Cash/Receivables/Payables", "S+P500 EMINI FUT  DEC24",
            "CASH COLLATERAL MSFUT USD", "USD CASH", "SSI US GOV MONEY MARKET CLASS"
        ])]

    # Normalize tickers and calculate weighted weights
    data[ticker_column] = data[ticker_column].apply(normalize_ticker)
    data['Weighted_Weight'] = data[weight_column] * etf_weight

    # Aggregate weights by ticker
    total_weights = data.groupby(ticker_column)[['Name', 'Weighted_Weight']].sum().reset_index()
    total_weights.columns = ['Ticker', 'Name', 'Total_Weight']
    total_weights = total_weights[total_weights['Total_Weight'] > 0]

    return total_weights

def calculate_total_weights(weights):
    """Calculate the total weights of stocks across multiple ETFs.

    This function downloads and processes holdings data for each ETF, aggregates the weights
    by ticker, and normalizes the total weights to sum to 1.

    Parameters
    ----------
    weights : list of float
        List of weights for each ETF in the portfolio.

    Returns
    -------
    pandas.DataFrame
        DataFrame with columns 'Symbol' and 'Weights', indexed by 'Symbol'.
    """
    urls = [
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPHQ",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPLV",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPMO",
        "https://www.ishares.com/us/products/239728/ishares-sp-500-value-etf/1467271812596.ajax?fileType=csv&fileName=IVE_holdings&dataType=fund",
        "https://www.ssga.com/us/en/intermediary/library-content/products/fund-data_smart_beta/etfs/us/holdings-daily-us-en-spyd.xlsx"
    ]

    final_data = pd.DataFrame()

    for url, weight in zip(urls, weights):
        etf_data = calculate_total_weights_per_etf(url, weight)
        if etf_data is not None:
            final_data = pd.concat([final_data, etf_data])

    if final_data.empty:
        return pd.DataFrame()

    # Aggregate weights by ticker
    final_data = final_data.groupby('Ticker', as_index=False)['Total_Weight'].sum()
    final_data['Total_Weight'] /= final_data['Total_Weight'].sum()

    # Sort data by weight in descending order
    # final_data.sort_values(by='Total_Weight', ascending=False, inplace=True)

    # Check for duplicate tickers
    duplicate_tickers = final_data[final_data.duplicated(subset='Ticker', keep=False)]
    if not duplicate_tickers.empty:
        print("Duplicate tickers found:")
        print(duplicate_tickers)

    # Save in a CSV file
    # final_data.to_csv("poids_actions_etf.csv", index=False)

    # Format the output
    final_data.columns = ['Symbol', 'Weights']
    final_data = final_data.set_index('Symbol')

    return final_data

#Stocks with several classes: fox and news corp

In [102]:
def calculate_portfolio_value(weights: pd.DataFrame) -> pd.DataFrame:
    """Calculate the weighted value of stocks in a portfolio using Yahoo Finance data.

    This function fetches the latest closing price for each stock symbol in the input DataFrame,
    calculates the weighted value based on the provided weights, and returns a summary DataFrame.

    Parameters
    ----------
    weights : pd.DataFrame
        A DataFrame indexed by stock symbols, with a 'Weights' column representing the portfolio weights.
        Example:
            ```
            Symbol  Weights
            AAPL    0.25
            MSFT    0.30
            GOOGL   0.45
            ```

    Returns
    -------
    pd.DataFrame
        A DataFrame with columns:
        - 'Symbol': Stock symbol.
        - 'Weight': Portfolio weight for the stock.
        - 'Last Close': Latest closing price from Yahoo Finance.
        - 'Weighted Value': Weighted value of the stock in the portfolio.

    Raises
    ------
    ValueError
        If the input DataFrame does not contain a 'Weights' column.
    """
    # Ensure the DataFrame has the expected structure
    if 'Weights' not in weights.columns:
        raise ValueError("The input DataFrame must have a 'Weights' column.")

    results = []

    for symbol, row in weights.iterrows():
        weight = row['Weights']

        try:
            # Fetch the stock data from Yahoo Finance
            stock = yf.Ticker(symbol)
            # Use '1d' period and ensure we get the most recent close
            hist = stock.history(period="1d")
            if hist.empty:
                print(f"No data available for {symbol}. Skipping...")
                continue
            last_close = hist['Close'].iloc[-1]

            # Calculate the weighted value
            weighted_value = weight * last_close
            results.append({
                'Symbol': symbol,
                'Weight': weight,
                'Last Close': last_close,
                'Weighted Value': weighted_value
            })
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            continue

    if not results:
        print("No valid data fetched for any symbol.")
        return pd.DataFrame()

    # Return results as a DataFrame
    return pd.DataFrame(results)

In [103]:
stock_weights = calculate_total_weights(weights_list[-1])
stock_weights.sort_values(by='Weights', ascending=False)

,Weights
Symbol,
AAPL,0.029413
AMZN,0.027313
NVDA,0.023637
MSFT,0.019348
META,0.018328
...,...
DVA,0.000073
NWS,0.000055
CZR,0.000055


In [104]:
calculate_portfolio_value(stock_weights)

$BFB: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


No data available for BFB. Skipping...


$BRKB: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


No data available for BRKB. Skipping...


$ESU5: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


No data available for ESU5. Skipping...


$XTSLA: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


No data available for XTSLA. Skipping...


,Symbol,Weight,Last Close,Weighted Value
0,A,0.000357,117.995003,0.042157
1,AAPL,0.029413,229.729996,6.757040
2,ABBV,0.004208,208.794998,0.878558
3,ABNB,0.001062,129.330002,0.137375
4,ABT,0.004442,131.460007,0.584001
...,...,...,...,...
463,XYZ,0.000302,79.070000,0.023904
464,YUM,0.002262,145.690002,0.329598
465,ZBH,0.000220,106.510002,0.023418
466,ZBRA,0.000608,319.959991,0.194578


## Adding constraints to the newly created ETF
### ESG constraints
We delete the worst 10% of the stocks in terms of ESG scores.

In [115]:
def esg_constraints(stock_weights: pd.DataFrame, quantile_threshold: float) -> pd.DataFrame:
    """Filter and reweight a portfolio based on ESG scores, keeping only the top quantile of stocks.

    This function fetches ESG scores for S&P 500 stocks, filters out stocks with poor ESG scores,
    and reweights the remaining stocks so their weights sum to 1.

    Parameters
    ----------
    stock_weights : pd.DataFrame
        DataFrame indexed by stock symbols, with a 'Weights' column representing the portfolio weights.
        Example:
            ```
            Symbol  Weights
            AAPL    0.25
            MSFT    0.30
            GOOGL   0.45
            ```
    quantile_threshold : float
        A float between 0 and 1. Only stocks with ESG scores better than this quantile are retained.
        For example, a value of 0.25 keeps the top 25% of stocks with the best ESG scores.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the filtered and reweighted stocks, with columns:
        - 'Symbol': Stock symbol.
        - 'Security': Stock name.
        - 'GICS Sector': Sector classification.
        - 'GICS Sub-Industry': Sub-industry classification.
        - 'ESG Score': ESG score of the stock.
        - 'Weights': Reweighted portfolio weights, normalized to sum to 1.

    Raises
    ------
    ValueError
        If `quantile_threshold` is not between 0 and 1.
    """
    # Validate quantile_threshold
    if not 0 <= quantile_threshold <= 1:
        raise ValueError("quantile_threshold must be between 0 and 1.")

    # Fetch S&P 500 tickers, names, sectors, and sub-industries
    try:
        response = requests.get(
            'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies',
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        response.raise_for_status()
        tables = pd.read_html(response.text)
        df_sp500 = tables[0][['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']]
        df_sp500 = df_sp500.set_index('Symbol')
        df_sp500.drop(index='GOOG', inplace=True)  # Drop GOOG (duplicate of GOOGL)
    except Exception as e:
        print(f"Error fetching S&P 500 data: {e}")
        return pd.DataFrame()

    # Fetch ESG scores for each ticker
    df_esg = df_sp500.copy()
    for ticker in df_sp500.index:
        try:
            # df_esg.loc[ticker, "ESG Score"] = yesg.get_historic_esg(ticker).iloc[-1,0] # yesg does not seem to work anymore
            esg_score = yf.Ticker(ticker).sustainability
            if esg_score is not None and "totalEsg" in esg_score.index and "esgScores" in esg_score.loc["totalEsg"]:
                df_esg.loc[ticker, "ESG Score"] = esg_score.loc["totalEsg", "esgScores"]
            else:
                df_esg.loc[ticker, "ESG Score"] = np.nan
                print(f"No ESG data available for {ticker}.")
        except Exception as e:
            df_esg.loc[ticker, "ESG Score"] = np.nan
            print(f"Error fetching ESG score for {ticker}: {e}")

    # Drop stocks with missing ESG scores
    df_esg.dropna(subset=["ESG Score"], inplace=True)

    # Keep only the top quantile of stocks based on ESG score
    # Note: Lower ESG scores are better (0 is the best)
    df_esg = df_esg[df_esg['ESG Score'] <= df_esg['ESG Score'].quantile(quantile_threshold)]

    # Merge with stock weights and reweight
    df_esg = df_esg.merge(stock_weights, left_index=True, right_index=True, how="inner")
    if df_esg.empty:
        print("No stocks remain after ESG filtering.")
        return pd.DataFrame()

    # Normalize weights to sum to 1
    df_esg['Weights'] /= df_esg['Weights'].sum()

    return df_esg

In [116]:
quantile_threshold = 0.9
df_esg = esg_constraints(stock_weights, quantile_threshold)
df_esg.sort_values(by='Weights', ascending=False)

/tmp/ipykernel_36101/642747978.py:49: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.

HTTP Error 404: 


No ESG data available for BRK.B.


HTTP Error 404: 


No ESG data available for BF.B.


HTTP Error 404: 


No ESG data available for FOX.


HTTP Error 404: 


No ESG data available for NWS.


,Security,GICS Sector,GICS Sub-Industry,ESG Score,Weights
Symbol,,,,,
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",18.88,0.032683
AMZN,Amazon,Consumer Discretionary,Broadline Retail,25.70,0.030350
NVDA,Nvidia,Information Technology,Semiconductors,12.10,0.026265
MSFT,Microsoft,Information Technology,Systems Software,16.91,0.021499
COST,Costco,Consumer Staples,Consumer Staples Merchandise Retail,29.02,0.016201
...,...,...,...,...,...
TECH,Bio-Techne,Health Care,Life Sciences Tools & Services,27.31,0.000092
WBA,Walgreens Boots Alliance,Consumer Staples,Drug Retail,15.37,0.000092
BLDR,Builders FirstSource,Industrials,Building Products,26.73,0.000092


### Sector constraints
We set a limit on the maximum weight of each sector in the portfolio.

In [117]:
def sector_constraints(df: pd.DataFrame, sector_threshold: float) -> pd.DataFrame:
    """Adjust portfolio weights to ensure no sector exceeds a specified weight threshold.

    This function iteratively reduces the weights of stocks in sectors that exceed the threshold,
    then redistributes the remaining weight proportionally to the other sectors.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing stock weights and sector information.
        Expected columns: 'GICS Sector' and 'Weights'.
        Example:
            ```
            Symbol  GICS Sector  Weights
            AAPL    Technology  0.25
            MSFT    Technology  0.30
            XOM     Energy      0.45
            ```
    sector_threshold : float
        Maximum allowed weight for any single sector in the portfolio (e.g., 0.2 for 20%).

    Returns
    -------
    pd.DataFrame
        DataFrame with adjusted weights, ensuring no sector exceeds the threshold.

    Raises
    ------
    ValueError
        If `sector_threshold` is not between 0 and 1.
    """
    # Validate sector_threshold
    if not 0 <= sector_threshold <= 1:
        raise ValueError("sector_threshold must be between 0 and 1.")

    # Calculate the current sector weights
    df_weights_by_sector = df[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()

    # Identify sectors exceeding the threshold
    sectors_above_threshold = df_weights_by_sector.loc[
        df_weights_by_sector['Weights'] > sector_threshold
    ].index

    # Iteratively adjust sector weights
    while len(sectors_above_threshold) > 0:
        for sector in sectors_above_threshold:
            # Reduce weights of stocks in the over-weighted sector
            df_sector = df[df['GICS Sector'] == sector]
            factor = sector_threshold / df_weights_by_sector.loc[sector, 'Weights']
            df.loc[df_sector.index, 'Weights'] *= factor

        # Recalculate sector weights after adjustment
        df_weights_by_sector = df[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()

        # Redistribute remaining weight to other sectors
        other_sectors = ~df['GICS Sector'].isin(sectors_above_threshold)
        current_sum_other_sectors = df.loc[other_sectors, 'Weights'].sum()
        target_sum_other_sectors = 1 - sector_threshold * len(sectors_above_threshold)

        if current_sum_other_sectors > 0:
            df.loc[other_sectors, 'Weights'] *= target_sum_other_sectors / current_sum_other_sectors

        # Update sectors above threshold
        sectors_above_threshold = df_weights_by_sector.loc[
            df_weights_by_sector['Weights'] > sector_threshold
        ].index

    return df

In [118]:
sector_threshold = 0.17
weights_after_sector = sector_constraints(df_esg, sector_threshold)

#Check that none of the sectors have a weight above the threshold
#weights_after_sector[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()

#Check that the sum of the weights is equal to 1
#weights_after_sector[['GICS Sector', 'Weights']].groupby('GICS Sector').sum().sum()

weights_after_sector.sort_values(by='Weights', ascending=False)

,Security,GICS Sector,GICS Sub-Industry,ESG Score,Weights
Symbol,,,,,
AMZN,Amazon,Consumer Discretionary,Broadline Retail,25.70,0.031212
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",18.88,0.030772
NVDA,Nvidia,Information Technology,Semiconductors,12.10,0.024729
MSFT,Microsoft,Information Technology,Systems Software,16.91,0.020242
COST,Costco,Consumer Staples,Consumer Staples Merchandise Retail,29.02,0.016661
...,...,...,...,...,...
TECH,Bio-Techne,Health Care,Life Sciences Tools & Services,27.31,0.000094
BLDR,Builders FirstSource,Industrials,Building Products,26.73,0.000094
GEN,Gen Digital,Information Technology,Systems Software,14.49,0.000086


### Stock constraints
We set a limit on the maximum weight of each stock in the portfolio.

In [119]:
def stock_constraints(df: pd.DataFrame, stock_threshold: float) -> pd.DataFrame:
    """Adjust portfolio weights to ensure no single stock exceeds a specified weight threshold.

    This function iteratively caps the weights of stocks that exceed the threshold,
    then redistributes the remaining weight proportionally to the other stocks.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing stock weights.
        Expected columns: 'Weights'.
        Example:
            ```
            Symbol  Weights
            AAPL    0.25
            MSFT    0.30
            XOM     0.45
            ```
    stock_threshold : float
        Maximum allowed weight for any single stock in the portfolio (e.g., 0.05 for 5%).

    Returns
    -------
    pd.DataFrame
        DataFrame with adjusted weights, ensuring no stock exceeds the threshold.

    Raises
    ------
    ValueError
        If `stock_threshold` is not between 0 and 1.
    """
    # Validate stock_threshold
    if not 0 <= stock_threshold <= 1:
        raise ValueError("stock_threshold must be between 0 and 1.")

    # Identify stocks exceeding the threshold
    stocks_above_threshold = df.loc[df['Weights'] > stock_threshold].index

    # Iteratively adjust stock weights
    while len(stocks_above_threshold) > 0:
        # Cap the weights of stocks above the threshold
        df.loc[stocks_above_threshold, 'Weights'] = stock_threshold

        # Redistribute remaining weight to other stocks
        other_stocks = ~df.index.isin(stocks_above_threshold)
        current_sum_other_stocks = df.loc[other_stocks, 'Weights'].sum()
        target_sum_other_stocks = 1 - stock_threshold * len(stocks_above_threshold)

        if current_sum_other_stocks > 0:
            df.loc[other_stocks, 'Weights'] *= target_sum_other_stocks / current_sum_other_stocks

        # Update stocks above threshold
        stocks_above_threshold = df.loc[df['Weights'] > stock_threshold].index

    return df

In [120]:
#print("Max weight for a stock before stock constraints: ", weights_after_sector['Weights'].max())
stock_threshold = 0.05
weights_after_stock = stock_constraints(weights_after_sector, stock_threshold)
#print("Max weight for a stock after stock constraints: ", weights_after_stock['Weights'].max())

#Check that none of the stocks have a weight above the threshold
#weights_after_stock['Weights'].max()

#Check that the sum of the weights is equal to 1
#weights_after_stock['Weights'].sum()

weights_after_stock.sort_values(by='Weights', ascending=False)

,Security,GICS Sector,GICS Sub-Industry,ESG Score,Weights
Symbol,,,,,
AMZN,Amazon,Consumer Discretionary,Broadline Retail,25.70,0.031212
AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",18.88,0.030772
NVDA,Nvidia,Information Technology,Semiconductors,12.10,0.024729
MSFT,Microsoft,Information Technology,Systems Software,16.91,0.020242
COST,Costco,Consumer Staples,Consumer Staples Merchandise Retail,29.02,0.016661
...,...,...,...,...,...
TECH,Bio-Techne,Health Care,Life Sciences Tools & Services,27.31,0.000094
BLDR,Builders FirstSource,Industrials,Building Products,26.73,0.000094
GEN,Gen Digital,Information Technology,Systems Software,14.49,0.000086


### Further constraints
We could have limited the universe by only taking into account stocks with a volume higher than a threshold but we think it is necessary because all S&P 500 stocks are liquid.

# References
### Implementations of equal risk contribution
- https://github.com/matthewgilbert/erc/blob/master/erc/erc.py
- https://github.com/mirca/riskparity.py (not used)
- https://thequantmba.wordpress.com/2016/12/14/risk-parityrisk-budgeting-portfolio-in-python/

### Papers
- [Paper of Maillard, Roncalli and Teiletche](http://thierry-roncalli.com/download/erc.pdf)
- [Slides of Maillard, Roncalli and Teiletche](http://www.thierry-roncalli.com/download/erc-slides.pdf)
- [Master's thesis of David Stefanovits](https://ethz.ch/content/dam/ethz/special-interest/math/risklab-dam/documents/walter-saxer-preis/ma-stefanovits.pdf)
